In [12]:
import os, sys
import numpy as np
import math as math
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('potts_tokenizer.py'))))
import nltk
import word2vec
import random
from utilities import Sentence, Product, AspectPattern
model = word2vec.load('/Users/tailin/SRS/Smart_Review_Summarization/srs/static_training_data/text8.bin')

In [13]:
#importing UI
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import pylab
from mpl_toolkits.mplot3d import Axes3D

In [14]:
def load_labeled_raw(file_name): 
    with open(file_name) as f:
        ls_list =[]
        for line in f.readlines():
            line_splitted = line.split('***')
            if len(line_splitted)==2:
                ls_list.append(Sentence(content=line_splitted[1],labeled_aspects=line_splitted[0]))
    return ls_list

def loadTrainingData(static_training_data_dir):
    import os
    sentences = []
    for data_file in os.listdir(static_training_data_dir):
        if data_file.endswith('labeled.txt'):
            sentences.extend(load_labeled_raw(os.path.join(static_training_data_dir, data_file)))
    return sentences

def load_labeled_aspects(file_name): 
    with open(file_name) as f:
        ls_list =[]
        for line in f.readlines():
            line_splitted = line.split('***')
            if len(line_splitted) == 2:
                line_splitted_2 = line_splitted[1].split('####')
                word2vec_features_list=[]
                if len(line_splitted_2) == 2:
                    all_feature_patterns=line_splitted_2[1][:-1]
                    all_feature_patterns_split=all_feature_patterns.split('###')
                    for feature_pattern in all_feature_patterns_split:
                        word2vec_feature=[]
                        feature_split=feature_pattern.split('##')
                        for feature in feature_split:
                            word2vec_feature.append(feature)      
                        word2vec_features_list.append(word2vec_feature)
                    ls_list.append(Sentence(content=line_splitted_2[0],labeled_aspects=line_splitted[0],word2vec_features_list=word2vec_features_list))
                elif len(line_splitted_2)==1:
                    ls_list.append(Sentence(content=line_splitted_2[0],labeled_aspects=line_splitted[0]))
    return ls_list

def savetofile(ls_list,filename):
    f=open(filename,'w+')
    for i in range(len(ls_list)):
        if ls_list[i].content[-1]=='\n':
            sentence=ls_list[i].content[:-1]
        else: sentence=ls_list[i].content

        if ls_list[i].word2vec_features_list:
            features_string='#'
            for word2vec_features in ls_list[i].word2vec_features_list:
                if word2vec_features:
                    features_string+='#'
                    for aspect in word2vec_features:
                        features_string+='##'+aspect
                else: features_string+='### '
            f.write(ls_list[i].labeled_aspects+"***"+sentence+features_string+'\n')
        else:
            f.write(ls_list[i].labeled_aspects+"***"+sentence+'\n')
    f.close()
    
def distill_dynamic(ls_list,aspectPatterns_list):
    for sentence in ls_list:
        if not sentence.pos_tagged_tokens:
            sentence.pos_tag()
        sentence.word2vec_matchDaynamicAspectPatterns(aspectPatterns_list)
        
def phrase2vec(phrase):
    vec_list=[]
    words=phrase.split(' ')
    for word in words:
        if word in model:
            vec=model[word]
            vec_list.append(vec)
    if vec_list:
        vec_whole=(np.sum(vec_list,axis=0))
        return vec_whole
    else:
        return np.zeros([100])
def find_similarity(word1,word2):
    word1_vec=phrase2vec(word1)
    word2_vec=phrase2vec(word2)
    similarity=np.dot(word1_vec,word2_vec)
    return similarity

In [15]:
static_aspect_list = ['price','pictures','video','zoom','size','design','battery','screen','detection','ease of use','quality','other features','no feature']
static_aspect_list_show= ['price','pictures','video','zoom','size','design','battery','screen','detection','ease of use','quality','useless']
static_aspect_list_fortraining = ['price','pictures','video','zoom','size','design','battery','screen','detection','ease use','quality']
num_useful=len(static_aspect_list_fortraining)
static_aspect_list_lookup={static_aspect_list[i]:i for i in range(num_useful)}
for i in range(num_useful,len(static_aspect_list)):
    static_aspect_list_lookup[static_aspect_list[i]]=num_useful
static_aspect_list_lookup['useless']=num_useful

static_aspect_vec_list=[]
for static_aspect in static_aspect_list_fortraining:
    splitted=static_aspect.split(' ')
    if len(splitted)==1:
        static_aspect_vec=model[static_aspect]
    elif len(splitted)>1:
        static_aspect_vec=np.zeros(100);
        for word in splitted:
            static_aspect_vec=np.add(static_aspect_vec,model[word])
        static_aspect_vec=static_aspect_vec/len(splitted)
    static_aspect_vec_list.append(static_aspect_vec)

In [5]:
from external.potts_tokenizer import PottsTokenizer
import nltk
import json
import string
import os
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer

def tokenize(string):
   """
   INPUT: string
   OUTPUT: a list of words
   """
   import re
   tokenizer = PottsTokenizer(preserve_case=False)
   token_list = tokenizer.tokenize(string)
   punctuation = re.compile(r'[-.?!,":;$*()|0-9]') # remove these punctuations and number 
   token_list = [punctuation.sub("", word) for word in token_list]  
   token_list = filter(None, token_list) #filters empty  

   #filter out stopwords 
   STOPWORDS = set(nltk.corpus.stopwords.words('english'))
   STOPWORDS.update(('nikon','would','does','got',"doesn't",'well'))
   token_list = [word for word in token_list if word not in STOPWORDS]

   #stemmer 
#    stemmer = PorterStemmer()
#    #stemmer = SnowballStemmer("english")
#    token_stem_list = [stemmer.stem(token) for token in token_list]

   return token_list

In [7]:
tokenize("The bottom line is that this 4 megapixel device takes impressively good pictures of such high detail resolution that my wife, a graphic designer, expects to use its output in PhotoShop for her professional work")

[u'bottom',
 u'line',
 u'megapixel',
 u'device',
 u'takes',
 u'impressively',
 u'good',
 u'pictures',
 u'high',
 u'detail',
 u'resolution',
 u'wife',
 u'graphic',
 u'designer',
 u'expects',
 u'use',
 u'output',
 u'photoshop',
 u'professional',
 u'work']

In [8]:
wordlist_dic={"battery": ["charge", "battery", "charger",  "external"], 
              "pictures": ["picture", "pictures",  "image", "clear"],
              "price": ["price", "expensive","cheap"],
              "zoom": ["zoom", "len", "optic"],
              "ease of use": ["easy", "use", "ease", "lightweight"], 
              "detection": ["focus", "shot",  "flash", "mode", "auto", "red","eye"], 
              "design": ["color", "size", "fast"], 
              "video": ["video", "movie", "record"], 
              "quality": [ "quality"], 
              "screen": ["screen", "touch"], 
              "size": [ "size", "lightweight", "easy", "carry", "fit", "compact"]}
static_wordlist_vec=[[] for i in range(num_useful)]
for i in range(num_useful):
    for j in range(len(wordlist_dic[static_aspect_list_show[i]])):
        word=wordlist_dic[static_aspect_list_show[i]][j]
        if word in model:
            static_wordlist_vec[i].append(model[word])

In [9]:
word_class_score={"battery": [["battery", 2.2866877529670733], ["batteries", 2.2866877529670733],["charge", 1.7749507146490977], 
             ["charger", 0.71840763848514966], ["external", 0.48147358360532011], ["stay", 0.44241455508516875], 
             ["camera", 0.40229996376026489], ["life", 0.3789028840812606], ["minute", 0.36867879590430724], 
             ["indicate", 0.36867879590430724], ["side", 0.35920381924257483]], 
 "pictures": [["picture", 1.0763675131135098], ["pictures", 1.0763675131135098], ["take", 0.73431061316412305], 
              ["good", 0.56983890410156657], ["easy", 0.43057481451068147], ["image", 0.42369542945450289],
              ["images", 0.42369542945450289],["pixel", 0.39745361492324693], ["quality", 0.39438257245861252], 
              ["use", 0.38221923084998854], ["great", 0.38040568900689425], ["zoom", 0.38040568900689425]], 
 "screen": [["screen", 2.2704285302825347], ["touch", 1.2756451597115113], ["view", 1.0759873281720536], 
            ["lcd", 0.93327501098874632], ["size", 0.78256197644681069], ["like", 0.7725620727789877], 
            ["awesome", 0.70290189305679174], ["terrific", 0.6976083079642923], 
            ["video", 0.65397319867891723], ["large", 0.64688764863666226]], 
 "zoom": [["zoom", 1.6775037614994346], ["len", 0.80346927851814021], ["great", 0.72540703199975554],
          ["optic", 0.59991125157497893], ["work", 0.55232742195528417], ["control", 0.51934530561447267], 
          ["x", 0.5108473930150681], ["love", 0.5050338757125854], ["vr", 0.49295244936973664], 
          ["well", 0.4626359131745682]], 
 "no feature": [["camera", 0.41836207930501079], ["one", 0.27465579777473037], ["feature", 0.26876394968429634], 
                ["use", 0.22280535304309729], ["purchase", 0.18368581712262524], ["take", 0.18204357819389122], 
                ["picture", 0.17638787772140088], ["canon", 0.17445892499647456], ["deal", 0.17255553553304437], 
                ["buy", 0.15703526108401913]],
 "detection": [["focus", 1.4575867589732454], ["flash", 0.82387811882963768], ["eye", 0.79634619915330374], 
               ["red", 0.77631649025962535], ["shot", 0.77073748568303657], ["mode", 0.73427317432424555], 
               ["auto", 0.69237412206931181], ["point", 0.65467695782717605], ["light", 0.5938571916109513],
               ["weak", 0.5868590448957447]], 
 "design": [["color", 1.4198562495381839], ["love", 1.0709688952611003], ["fast", 0.93611300773017336], 
            ["xma", 0.80340078541901216], ["delight", 0.80340078541901216], ["attract", 0.78073156779735653], 
            ["surprise", 0.76109459829375026], ["ever", 0.76109459829375026], ["purple", 0.67879815889148232],
            ["wife", 0.64184015473252554]], 
 "video": [["video", 2.3987633532570563], ["movie", 1.5349618564834437], ["clip", 1.3457260983542545],
           ["record", 0.92329994104730861], ["take", 0.8357243551356387], ["really", 0.7918375056264082],
           ["picture", 0.77653927438329107], ["definition", 0.63352037442769149], ["good", 0.57522490508551338], 
           ["high", 0.55554163519869737]], 
 "ease of use": [["easy", 1.6159524665069551], ["use", 1.1741635375574053], ["ease", 0.6321657802793097], 
                 ["picture", 0.58686976924914669], ["take", 0.55580704100587053], ["appreciate", 0.4874853236376982], 
                 ["telephoto", 0.4874853236376982], ["adjust", 0.46814356985821376], 
                 ["well", 0.46646798775658654], ["lightweight", 0.45997164985240774]], 
 "quality": [["quality", 1.0285728629689821], ["feel", 0.99471826867174573], ["corrupt", 0.86853029194511344], 
             ["click", 0.86853029194511344], ["turn", 0.81166504426014696], ["flip", 0.79622204129192342], 
             ["one", 0.74910869060686192], ["really", 0.70047163959259184], ["memory", 0.67146354648941131], 
             ["problem", 0.64774805728521767]], 
 "price": [["price", 2.5079155102234743], ["like", 1.2729715971926501], ["awesome", 1.235403327190725],
           ["also", 1.2083851554010951], ["touch", 1.0191104581717307], ["screen", 0.782441204731949],
           ["capable", 0.5511314677545609], ["great", 0.4952826421134694], ["good", 0.46773338241801843], 
           ["zoom", 0.42452797895440236]], 
 "size": [["size", 0.9678450326349527], ["lightweight", 0.91105748374175766], ["carry", 0.88260306747798012], 
          ["fit", 0.81834619728397018], ["compact", 0.73886062395845831], ["small", 0.63887288047850199], 
          ["easy", 0.59876810142891645], ["pocket", 0.53662254303255053], ["amazing", 0.50481280734791878], 
          ["anywhere", 0.49522997170832922]]}

In [3]:
aspectPatterns_list=[]

# define an aspect_patterns_1
aspectPatterns_1 = []
pattern_name = 'adj_nn'
pattern_structure ="""
adj_nn:{<JJ><NN.?>}
"""
aspectTagIndices = [0,1]
aspectPattern = AspectPattern(name='adj_nn', structure=pattern_structure, aspectTagIndices=aspectTagIndices)
aspectPatterns_1.append(aspectPattern)

pattern_name = 'nn_nn'
pattern_structure ="""
nn_nn:{<NN.?><NN.?>}
"""
aspectTagIndices = [0,1]
aspectPattern = AspectPattern(name='nn_nn', structure=pattern_structure, aspectTagIndices=aspectTagIndices)
aspectPatterns_1.append(aspectPattern)
aspectPatterns_list.append(aspectPatterns_1)

# define an aspect_patterns_2
aspectPatterns_2 = []
pattern_name = 'nn'
pattern_structure ="""
nn:{<NN.?>}
"""
aspectTagIndices = [0]
aspectPattern = AspectPattern(name='nn', structure=pattern_structure, aspectTagIndices=aspectTagIndices)
aspectPatterns_2.append(aspectPattern)
aspectPatterns_list.append(aspectPatterns_2)

# define an aspect_patterns_3
aspectPatterns_3 = []
pattern_name = 'adj'
pattern_structure ="""
adj:{<JJ.?>}
"""
aspectTagIndices = [0]
aspectPattern = AspectPattern(name='adj', structure=pattern_structure, aspectTagIndices=aspectTagIndices)
aspectPatterns_3.append(aspectPattern)
aspectPatterns_list.append(aspectPatterns_3)

# define an aspect_patterns_4
aspectPatterns_4 = []
pattern_name = 'adv_compara'
pattern_structure ="""
adv_compara:{<RBR>}
"""
aspectTagIndices = [0]
aspectPattern = AspectPattern(name='adv_compara', structure=pattern_structure, aspectTagIndices=aspectTagIndices)
aspectPatterns_4.append(aspectPattern)

pattern_name = 'adv_super'
pattern_structure ="""
adv_super:{<RBS>}
"""
aspectTagIndices = [0]
aspectPattern = AspectPattern(name='adv_super', structure=pattern_structure, aspectTagIndices=aspectTagIndices)
aspectPatterns_4.append(aspectPattern)
aspectPatterns_list.append(aspectPatterns_4)

In [4]:
# ls_list = load_labeled_raw('static_training_data/training.txt')
# test_list = load_labeled_raw('static_training_data/testing.txt')
# distill_dynamic(ls_list,aspectPatterns_list)
# distill_dynamic(test_list,aspectPatterns_list)
# savetofile(ls_list,'static_training_data/allsentences_all_nn.txt')
# savetofile(test_list,'static_training_data/testing_all_nn.txt')

In [5]:
# ls_list=loadTrainingData('static_training_data/labeled_data')

In [8]:
ls_list = load_labeled_aspects('static_training_data/training_all_nn.txt')
test_list = load_labeled_aspects('static_training_data/testing_all_nn.txt')

In [9]:
for i in range(3):
    print ls_list[i].content
    print ls_list[i].word2vec_features_list
    print '\n'

The Nikon Coolpix S6400 is very small, measuring 3.8 x 2.4 x 1.1 inches and weighs about 5 ounces.
[['nikon coolpix', 'coolpix s6400'], ['coolpix', 's6400', 'inches', 'ounces'], ['nikon', 'small', 'x', 'x'], [' ']]


The 12X lens extends from a 35 mm equivalent focal length of 25 mm to 300 mm.
[['focal length'], ['lens', 'mm', 'length', 'mm', 'mm'], ['equivalent', 'focal'], [' ']]


The 1/2.3 inch image sensor is typical of digital cameras of this class.
[['inch image', 'digital cameras', 'image sensor'], ['image', 'sensor', 'cameras', 'class'], ['inch', 'typical', 'digital'], [' ']]




In [15]:
def sentence2vec(sentence, mode, aspect_pattern_mode, mini_threshold=0, ratio_threshold=0):
    vec_list=[]
    num_static_aspect=len(static_aspect_vec_list)
    word2vec_features=[]
    for aspect_k in aspect_pattern_mode:
        if aspect_k<len(sentence.word2vec_features_list):   
            word2vec_features=word2vec_features+sentence.word2vec_features_list[aspect_k]
    if word2vec_features:
        for dynamic_aspect in word2vec_features:
            dynamic_aspect_splitted=dynamic_aspect.split(' ')
            aspect_phrase_vec=[]
            for aspect in dynamic_aspect_splitted:
                if aspect in model:
                    aspect_word_vec=model[aspect]
                    aspect_phrase_vec.append(aspect_word_vec)
            if aspect_phrase_vec:
                vec_list.append(aspect_phrase_vec)    
        #calculating vector distance of the dynamic aspects to each static aspect:
    if vec_list:
        similarity_matrix=np.zeros([len(vec_list),num_static_aspect])
        for i in range(len(vec_list)):
            for j in range(num_static_aspect):
                similarity_list=[]
                for vec_word in vec_list[i]:
                    word_similarity=np.dot(vec_word,static_aspect_vec_list[j])
                    similarity_list.append(word_similarity)
                if mode == 1 or mode == 3:
                    similarity_item = np.max(similarity_list)
                if mode == 2:
                    similarity_item = np.sum(similarity_list)
                similarity_matrix[i][j]=similarity_item
        
        similarity_catagory=[0 for j in range(num_static_aspect)]
        if mode == 1:
            for j in range(num_static_aspect):
                similarity_catagory[j]=np.max(similarity_matrix[:,j])
        if mode == 2:
            for j in range(num_static_aspect):
                similarity_catagory[j]=np.sum(similarity_matrix[:,j])
        if mode == 3:
            similarity_maxmatching_matrix=np.zeros([len(vec_list),num_static_aspect])
            for i in range(len(vec_list)):
                k=0
                max_k=0
                for j in range(num_static_aspect):
                    if similarity_matrix[i,j]>max_k:
                        k=j
                        max_k=similarity_matrix[i,j]
                similarity_maxmatching_matrix[i,k]=max_k
            for j in range(num_static_aspect):
                similarity_catagory[j]=np.sum(similarity_maxmatching_matrix[:,j])
        similarity_list_sorted,static_aspect_list_sorted = (list(x) for x in zip(*sorted(zip(similarity_catagory, static_aspect_list), reverse=True)))
        sim_whole_list_sorted=[]
        for i in range(num_static_aspect):
            item=(static_aspect_list_sorted[i],round(similarity_list_sorted[i],4))
            sim_whole_list_sorted.append(item)
        if sim_whole_list_sorted[0][1]>mini_threshold:
            if sim_whole_list_sorted[1][1]!=0 and sim_whole_list_sorted[0][1]/sim_whole_list_sorted[1][1]>=ratio_threshold:
                prediction=sim_whole_list_sorted[0][0]
            elif sim_whole_list_sorted[1][1]==0 and sim_whole_list_sorted[0][1]!=0:
                prediction=sim_whole_list_sorted[0][0]
            else: prediction='useless'
        else: prediction='useless'
        return (prediction,sim_whole_list_sorted,sentence.word2vec_features_list)
    else:
        return ('useless',[],[])

In [16]:
A=np.array([1, 3, 2, 4])
B=np.array([3, 2, 8, 3])
C=np.array([2, 1, 5, 4])
D=np.array([1, 1, 5, 4])
E=np.matrix('2 5; 6 7')
np.sum(E,axis=1)
np.mean(E)

5.0

In [17]:
def sentence2vec_test(sentence, mode, aspect_pattern_mode, mini_threshold=0, ratio_threshold=0):
    vec_list=[]
    num_static_aspect=len(static_aspect_vec_list)
    word2vec_features=[]
    for aspect_k in aspect_pattern_mode:
        if aspect_k<len(sentence.word2vec_features_list):   
            word2vec_features=word2vec_features+sentence.word2vec_features_list[aspect_k]
    if word2vec_features:
        for dynamic_aspect in word2vec_features:
            dynamic_aspect_splitted=dynamic_aspect.split(' ')
            aspect_phrase_vec=[]
            for aspect in dynamic_aspect_splitted:
                if aspect in model:
                    aspect_word_vec=model[aspect]
                    aspect_phrase_vec.append(aspect_word_vec)
            if aspect_phrase_vec:
                vec_list.append(aspect_phrase_vec)    
        #calculating vector distance of the dynamic aspects to each static aspect:
    nearest_word=(-1,'',-1)
    if vec_list:
        similarity_matrix=np.zeros([len(vec_list),num_static_aspect])
        for i in range(len(vec_list)):
            for j in range(num_static_aspect):
                similarity_list=[]
                for vec_word in vec_list[i]:
                    word_similarity=np.dot(vec_word,static_aspect_vec_list[j])
                    similarity_list.append(word_similarity)
                if mode == 1 or mode == 3:
                    similarity_item = np.max(similarity_list)
                if mode == 2:
                    similarity_item = np.sum(similarity_list)
                similarity_matrix[i][j]=similarity_item
        
        item_max=np.argmax(similarity_matrix)
        item_i=item_max/num_static_aspect
        item_j=item_max-item_i*num_static_aspect
        
        if static_aspect_list[item_j]==sentence.labeled_aspects:
            nearest_word=(item_j,word2vec_features[item_i],similarity_matrix[item_i][item_j])    
        
        similarity_catagory=[0 for j in range(num_static_aspect)]
        if mode == 1:
            for j in range(num_static_aspect):
                similarity_catagory[j]=np.max(similarity_matrix[:,j])
        if mode == 2:
            for j in range(num_static_aspect):
                similarity_catagory[j]=np.sum(similarity_matrix[:,j])
        if mode == 3:
            similarity_maxmatching_matrix=np.zeros([len(vec_list),num_static_aspect])
            for i in range(len(vec_list)):
                k=0
                max_k=0
                for j in range(num_static_aspect):
                    if similarity_matrix[i,j]>max_k:
                        k=j
                        max_k=similarity_matrix[i,j]
                similarity_maxmatching_matrix[i,k]=max_k
            for j in range(num_static_aspect):
                similarity_catagory[j]=np.sum(similarity_maxmatching_matrix[:,j])
        similarity_list_sorted,static_aspect_list_sorted = (list(x) for x in zip(*sorted(zip(similarity_catagory, static_aspect_list), reverse=True)))
        sim_whole_list_sorted=[]
        for i in range(num_static_aspect):
            item=(static_aspect_list_sorted[i],round(similarity_list_sorted[i],4))
            sim_whole_list_sorted.append(item)
        if sim_whole_list_sorted[0][1]>mini_threshold:
            if sim_whole_list_sorted[1][1]!=0 and sim_whole_list_sorted[0][1]/sim_whole_list_sorted[1][1]>=ratio_threshold:
                prediction=sim_whole_list_sorted[0][0]
            elif sim_whole_list_sorted[1][1]==0 and sim_whole_list_sorted[0][1]!=0:
                prediction=sim_whole_list_sorted[0][0]
            else: prediction='useless'
        else: prediction='useless'
        return (prediction,sim_whole_list_sorted,sentence.word2vec_features_list,nearest_word,sentence.labeled_aspects,sentence.content)
    else:
        return ('useless',[],[],nearest_word,sentence.labeled_aspects,sentence.content)

In [18]:
ccount=0
nearest_word_list=[[] for i in range(len(static_aspect_list_fortraining))]
for i in range(len(ls_list)):
    ans=sentence2vec_test(ls_list[i], 1, [0,1,2], mini_threshold=0.8, ratio_threshold=0.9)
    if abs(int(ans[3][0])+1)>0.01:
        nearest_word_list[ans[3][0]].append(ans[3][1])
#         print ans
#         print '\n'
        ccount+=1
print ccount
for i in range(len(nearest_word_list)):
    print (i,static_aspect_list[i])
    print nearest_word_list[i]
    print '\n'

262
(0, 'price')
['great price', 'great price', 'price', 'low price', 'price', 'sale', 'price', 'price']


(1, 'pictures')
['shots', ' ', 'great photos', 'decent pictures', 'pictures', 'multiple pictures', 'pictures', 'good pictures', 'nice pictures', 'fabulous pictures', 'pictures', 'pictures', 'pictures', 'pictures', 'good pictures', 'pictures', 'good pictures', 'pictures', 'good pictures', 'pictures', 'good pictures', 'photos', 'action photos', 'pictures', 'good pictures', 'photos', 'photos', 'photos', 'photos', 'photos', 'photos', 'pictures', 'pictures', 'pictures', 'pictures', 'pictures', 'good pictures', 'pictures', 'pictures', 'good pictures', 'pictures', 'pictures', 'pictures', 'sharp photos', 'photos', 'pictures', 'marvelous pictures', 'pleasing pictures', 'pictures', 'daytime pictures', 'clear pictures', 'clear pictures', 'pictures', 'pictures', 'good pictures', 'images', 'pictures', 'photos', 'share pictures', 'beautiful pictures', 'pictures', 'sony', 'pictures', 'pictures',

In [19]:
dimen=[0,1,20]
data_range=range(1000)
scattered=[[] for j in range(len(dimen))]
for i in data_range:
    for j in range(len(dimen)):
        scattered[j].append(model.vectors[i][dimen[j]])

fig = pylab.figure()
ax = Axes3D(fig)
ax.scatter(scattered[0], scattered[1], scattered[2],s=1.5)
plt.show()

In [19]:

dimen=[10,36,47]
num_useful=len(static_aspect_list_fortraining)
all_vec_list=[[] for i in range(num_useful)]
dim_vec_list=[[[] for j in range(len(dimen))] for i in range(num_useful) ]
for i in range(num_useful):    
    for nearest_words in nearest_word_list[i]:
        nearest_words_split=nearest_words.split(' ')
        for word in nearest_words_split:
            if word in model:
                all_vec_list[i].append(model[word])
                for j in range(len(dimen)):
                    dim_vec_list[i][j].append(model[word][dimen[j]])
                    
fig = pylab.figure()
ax = Axes3D(fig)
palette=range(num_useful)
random.shuffle(palette)
for i in range(3):
    color=cm.rainbow(1.*palette[i]/num_useful)
    ax.scatter(dim_vec_list[i][0],dim_vec_list[i][1],dim_vec_list[i][2],c=color)
plt.show()

In [20]:
def testing(ls_list,mode,aspect_pattern_mode,useful_threshold,peak_ratio_threshold,lookup,Isprint=0):
    #aspect_pattern_mode:  0:adj_nn, nn_nn 1:nn 2:adj 3:adv_compa, adv_super
    #mode:  1:both max  2: both sum  3:use only the best match for a word

    classification_list=[]
    useful_list=[]
    num_useful=len(static_aspect_list_fortraining)
    count=0
    count_correct=0
    count_useless_true=0
    count_useless_prediction=0
    count_useful_correct=0
    correctness_matrix=np.zeros([num_useful+1,num_useful+1],dtype=np.int)
    classified_sentences_true=[[] for i in range(num_useful+1)]
    classified_sentences_predict=[[] for i in range(num_useful+1)]
    classified_sentences_matrix=[[[] for j in range(num_useful+1)] for i in range(num_useful+1)]
    ii=0
    for sentence in ls_list:
        #if sentence.dynamic_aspects:                
        count=count+1
        aspect_prediction=sentence2vec(sentence,mode,aspect_pattern_mode,useful_threshold,peak_ratio_threshold)
        aspect_true=sentence.labeled_aspects
        classification=(aspect_true,aspect_prediction[0],sentence.word2vec_features_list,aspect_prediction[1][:3],sentence.content)

        if static_aspect_list_lookup[aspect_true]!=11:
            useful_list.append(ii)
        ii+=1
        classification_list.append(classification)
        correctness_matrix[static_aspect_list_lookup[aspect_true],static_aspect_list_lookup[aspect_prediction[0]]]+=1
        classified_sentences_true[static_aspect_list_lookup[aspect_true]].append(classification)
        classified_sentences_predict[static_aspect_list_lookup[aspect_prediction[0]]].append(classification)
        classified_sentences_matrix[static_aspect_list_lookup[aspect_true]][static_aspect_list_lookup[aspect_prediction[0]]].append(classification)
    
    count_useful_total=sum(sum(correctness_matrix[:num_useful,:]))
    count_useful_prediction_total=sum(sum(correctness_matrix[:,:num_useful]))
    count_useful_correct=sum(correctness_matrix.diagonal()[0:num_useful])
    count_correct=sum(correctness_matrix.diagonal())
    count_useless2useful=sum(sum(correctness_matrix[num_useful:num_useful+1,0:num_useful]))
    count_useful2useless=sum(sum(correctness_matrix[0:num_useful,num_useful:num_useful+1]))
    count_item_prediction=np.sum(correctness_matrix,axis=0)
    count_item_truelabel=np.sum(correctness_matrix,axis=1)
    correct_all=round(1.*count_correct/count,3)
    if abs(count_useful_prediction_total)>0.01:
        correct_useful_precision=round(1.*count_useful_correct/count_useful_prediction_total,3)
    else: correct_useful_precision=-1
    correct_useful_recall=round(1.*count_useful_correct/count_useful_total,3)
    if Isprint==1:
        correct_item_precision=np.zeros(num_useful+1)
        for ii in range(num_useful+1):
            if abs(count_item_prediction[ii])>0.01:
                correct_item_precision[ii]=np.divide(1.*correctness_matrix.diagonal()[ii],count_item_prediction[ii])
            else: correct_item_precision[ii]=-1
        correct_item_recall=np.divide(1.*correctness_matrix.diagonal(),count_item_truelabel)    
        print ((count,str(round(100.*correct_all,2))+'%'),(count_useful_total,str(round(100.*correct_useful_precision))+'%',str(round(100.*correct_useful_recall))+'%'),count_useless2useful,count_useful2useless)
        for i in range(num_useful+1):
            print (static_aspect_list_show[i],round(correct_item_precision[i],3),round(correct_item_recall[i],3))
        print static_aspect_list_show
        print correctness_matrix
        for i in range(len(classified_sentences_matrix[lookup[0]][lookup[1]])):
            print classified_sentences_matrix[lookup[0]][lookup[1]][i]
            print " "
    return (correct_all,correct_useful_precision,correct_useful_recall,count_useless2useful,count_useful2useless,useful_list)

In [21]:
testing(ls_list,1,[0,1,2],0.85,0.9,[8,11],1)

((1155, '64.5%'), (566, '51.0%', '42.0%'), 79, 184)
('price', 0.667, 0.267)
('pictures', 0.493, 0.472)
('video', 0.75, 0.6)
('zoom', 0.587, 0.557)
('size', 0.588, 0.317)
('design', -1.0, 0.0)
('battery', 0.656, 0.646)
('screen', 0.444, 0.816)
('detection', -1.0, 0.0)
('ease of use', -1.0, 0.0)
('quality', 0.059, 0.222)
('useless', 0.735, 0.866)
['price', 'pictures', 'video', 'zoom', 'size', 'design', 'battery', 'screen', 'detection', 'ease of use', 'quality', 'useless']
[[  8   2   0   5   0   0   0  10   0   0   1   4]
 [  0  67   3  15   2   0   0  11   0   0  19  25]
 [  0   5  12   1   0   0   0   0   0   0   0   2]
 [  0   2   0  44   0   0   0  10   0   0   0  23]
 [  0   8   0   3  20   0   0   3   0   0   0  29]
 [  0   0   0   0   6   0   0   0   0   0   1  24]
 [  0   1   0   0   0   0  42   3   0   0   0  19]
 [  0   4   0   0   0   0   0  40   0   0   1   4]
 [  0   3   0   0   0   0   0   0   0   0   0  16]
 [  0  15   0   1   0   0   0  10   0   0   0  33]
 [  0   1   0  

(0.645,
 0.51,
 0.415,
 79,
 184,
 [0,
  1,
  2,
  4,
  5,
  6,
  7,
  8,
  11,
  12,
  14,
  22,
  23,
  24,
  25,
  26,
  31,
  32,
  34,
  37,
  38,
  42,
  45,
  48,
  49,
  50,
  51,
  52,
  54,
  55,
  56,
  57,
  61,
  63,
  64,
  66,
  67,
  74,
  76,
  80,
  82,
  86,
  89,
  91,
  92,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  109,
  111,
  112,
  113,
  114,
  115,
  117,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  129,
  133,
  134,
  137,
  138,
  139,
  140,
  141,
  143,
  147,
  148,
  151,
  152,
  153,
  154,
  155,
  156,
  157,
  158,
  159,
  161,
  165,
  166,
  169,
  170,
  171,
  172,
  174,
  175,
  176,
  177,
  178,
  179,
  180,
  181,
  183,
  187,
  188,
  191,
  192,
  193,
  194,
  196,
  197,
  198,
  200,
  201,
  203,
  204,
  205,
  206,
  207,
  209,
  213,
  214,
  217,
  218,
  219,
  220,
  222,
  223,
  224,
  226,
  227,
  229,
  230,
  231,
  233,
  234,
  235,
  236,
  237,
  239,
  243,
  244,
  247,
  248,

In [22]:
def sentence2vec_list(sentence, mode, item_mode,aspect_pattern_mode, mini_threshold=0, ratio_threshold=0):
    vec_list=[]
    num_static_aspect=len(static_aspect_vec_list)
    word2vec_features=[]
    for aspect_k in aspect_pattern_mode:
        if aspect_k<len(sentence.word2vec_features_list):   
            word2vec_features=word2vec_features+sentence.word2vec_features_list[aspect_k]
    if word2vec_features:
        for dynamic_aspect in word2vec_features:
            dynamic_aspect_splitted=dynamic_aspect.split(' ')
            aspect_phrase_vec=[]
            for aspect in dynamic_aspect_splitted:
                if aspect in model:
                    aspect_word_vec=model[aspect]
                    aspect_phrase_vec.append(aspect_word_vec)
            if aspect_phrase_vec:
                vec_list.append(aspect_phrase_vec)    
        #calculating vector distance of the dynamic aspects to each static aspect:
    if vec_list:
        similarity_matrix=np.zeros([len(vec_list),num_static_aspect])
        for i in range(len(vec_list)):
            for j in range(num_static_aspect):               
                similarity_item_matrix=np.zeros([len(vec_list[i]),len(static_wordlist_vec[j])])
                for kk in range(len(vec_list[i])):
                    for ll in range(len(static_wordlist_vec[j])):
                        similarity_item_matrix[kk][ll]=np.dot(vec_list[i][kk],static_wordlist_vec[j][ll])
                if item_mode==1:
                    similarity_item=np.max(similarity_item_matrix)
                elif item_mode==2:
                    similarity_item_row=np.max(similarity_item_matrix,axis=1)
                    similarity_item=np.sum(similarity_item_row)
                elif item_mode==3:
                    similarity_item_column=np.mean(similarity_item_matrix,axis=0)
                    similarity_item=np.max(similarity_item_column)
                elif item_mode==4:
                    similarity_item=np.mean(similarity_item_matrix)
                similarity_matrix[i][j]=similarity_item
        
        similarity_catagory=[0 for j in range(num_static_aspect)]
        if mode == 1:
            for j in range(num_static_aspect):
                similarity_catagory[j]=np.max(similarity_matrix[:,j])
        if mode == 2:
            for j in range(num_static_aspect):
                similarity_catagory[j]=np.sum(similarity_matrix[:,j])
        if mode == 3:
            similarity_maxmatching_matrix=np.zeros([len(vec_list),num_static_aspect])
            for i in range(len(vec_list)):
                k=0
                max_k=0
                for j in range(num_static_aspect):
                    if similarity_matrix[i,j]>max_k:
                        k=j
                        max_k=similarity_matrix[i,j]
                similarity_maxmatching_matrix[i,k]=max_k
            for j in range(num_static_aspect):
                similarity_catagory[j]=np.sum(similarity_maxmatching_matrix[:,j])
        similarity_list_sorted,static_aspect_list_sorted = (list(x) for x in zip(*sorted(zip(similarity_catagory, static_aspect_list), reverse=True)))
        sim_whole_list_sorted=[]
        for i in range(num_static_aspect):
            item=(static_aspect_list_sorted[i],round(similarity_list_sorted[i],4))
            sim_whole_list_sorted.append(item)
        if sim_whole_list_sorted[0][1]>mini_threshold:
            if sim_whole_list_sorted[1][1]!=0 and sim_whole_list_sorted[0][1]/sim_whole_list_sorted[1][1]>=ratio_threshold:
                prediction=sim_whole_list_sorted[0][0]
            elif sim_whole_list_sorted[1][1]==0 and sim_whole_list_sorted[0][1]!=0:
                prediction=sim_whole_list_sorted[0][0]
            else: prediction='useless'
        else: prediction='useless'
        return (prediction,sim_whole_list_sorted,sentence.word2vec_features_list)
    else:
        return ('useless',[],[])

In [23]:
def testing_test(ls_list,mode,item_mode,aspect_pattern_mode,useful_threshold,peak_ratio_threshold,lookup,Isprint=0):
    #aspect_pattern_mode:  0:adj_nn, nn_nn 1:nn 2:adj 3:adv_compa, adv_super
    #mode:  1:both max  2: both sum  3:use only the best match for a word

    classification_list=[]
    num_useful=len(static_aspect_list_fortraining)
    count=0
    count_correct=0
    count_useless_true=0
    count_useless_prediction=0
    count_useful_correct=0
    correctness_matrix=np.zeros([num_useful+1,num_useful+1],dtype=np.int)
    classified_sentences_true=[[] for i in range(num_useful+1)]
    classified_sentences_predict=[[] for i in range(num_useful+1)]
    classified_sentences_matrix=[[[] for j in range(num_useful+1)] for i in range(num_useful+1)]
    for sentence in ls_list:
        #if sentence.dynamic_aspects:
        count=count+1
        aspect_prediction=sentence2vec_list(sentence,mode,item_mode,aspect_pattern_mode,useful_threshold,peak_ratio_threshold)
        aspect_true=sentence.labeled_aspects
        classification=(aspect_true,aspect_prediction[0],sentence.word2vec_features_list,aspect_prediction[1][:3],sentence.content)

        #print classification
        classification_list.append(classification)
        correctness_matrix[static_aspect_list_lookup[aspect_true],static_aspect_list_lookup[aspect_prediction[0]]]+=1
        classified_sentences_true[static_aspect_list_lookup[aspect_true]].append(classification)
        classified_sentences_predict[static_aspect_list_lookup[aspect_prediction[0]]].append(classification)
        classified_sentences_matrix[static_aspect_list_lookup[aspect_true]][static_aspect_list_lookup[aspect_prediction[0]]].append(classification)
    
    count_useful_total=sum(sum(correctness_matrix[:num_useful,:]))
    count_useful_prediction_total=sum(sum(correctness_matrix[:,:num_useful]))
    count_useful_correct=sum(correctness_matrix.diagonal()[0:num_useful])
    count_correct=sum(correctness_matrix.diagonal())
    count_useless2useful=sum(sum(correctness_matrix[num_useful:num_useful+1,0:num_useful]))
    count_useful2useless=sum(sum(correctness_matrix[0:num_useful,num_useful:num_useful+1]))
    count_item_prediction=np.sum(correctness_matrix,axis=0)
    count_item_truelabel=np.sum(correctness_matrix,axis=1)
    correct_all=round(1.*count_correct/count,3)
    if abs(count_useful_prediction_total)>0.01:
        correct_useful_precision=round(1.*count_useful_correct/count_useful_prediction_total,3)
    else: correct_useful_precision=-1
    correct_useful_recall=round(1.*count_useful_correct/count_useful_total,3)
    if Isprint==1:
        correct_item_precision=np.zeros(num_useful+1)
        for ii in range(num_useful+1):
            if abs(count_item_prediction[ii])>0.01:
                correct_item_precision[ii]=np.divide(1.*correctness_matrix.diagonal()[ii],count_item_prediction[ii])
            else: correct_item_precision[ii]=-1
        correct_item_recall=np.divide(1.*correctness_matrix.diagonal(),count_item_truelabel)    
        print ((count,str(round(100.*correct_all,2))+'%'),(count_useful_total,str(round(100.*correct_useful_precision))+'%',str(round(100.*correct_useful_recall))+'%'),count_useless2useful,count_useful2useless)
        for i in range(num_useful+1):
            print (static_aspect_list_show[i],round(correct_item_precision[i],3),round(correct_item_recall[i],3))
        print static_aspect_list_show
        print correctness_matrix
        for i in range(len(classified_sentences_matrix[lookup[0]][lookup[1]])):
            print classified_sentences_matrix[lookup[0]][lookup[1]][i]
            print " "
    return (correct_all,correct_useful_precision,correct_useful_recall,count_useless2useful,count_useful2useless)

In [24]:
testing(ls_list,1,[0,1,2],0.85,0.9,[11,6],1)

((1155, '64.5%'), (566, '51.0%', '42.0%'), 79, 184)
('price', 0.667, 0.267)
('pictures', 0.493, 0.472)
('video', 0.75, 0.6)
('zoom', 0.587, 0.557)
('size', 0.588, 0.317)
('design', -1.0, 0.0)
('battery', 0.656, 0.646)
('screen', 0.444, 0.816)
('detection', -1.0, 0.0)
('ease of use', -1.0, 0.0)
('quality', 0.059, 0.222)
('useless', 0.735, 0.866)
['price', 'pictures', 'video', 'zoom', 'size', 'design', 'battery', 'screen', 'detection', 'ease of use', 'quality', 'useless']
[[  8   2   0   5   0   0   0  10   0   0   1   4]
 [  0  67   3  15   2   0   0  11   0   0  19  25]
 [  0   5  12   1   0   0   0   0   0   0   0   2]
 [  0   2   0  44   0   0   0  10   0   0   0  23]
 [  0   8   0   3  20   0   0   3   0   0   0  29]
 [  0   0   0   0   6   0   0   0   0   0   1  24]
 [  0   1   0   0   0   0  42   3   0   0   0  19]
 [  0   4   0   0   0   0   0  40   0   0   1   4]
 [  0   3   0   0   0   0   0   0   0   0   0  16]
 [  0  15   0   1   0   0   0  10   0   0   0  33]
 [  0   1   0  

(0.645,
 0.51,
 0.415,
 79,
 184,
 [0,
  1,
  2,
  4,
  5,
  6,
  7,
  8,
  11,
  12,
  14,
  22,
  23,
  24,
  25,
  26,
  31,
  32,
  34,
  37,
  38,
  42,
  45,
  48,
  49,
  50,
  51,
  52,
  54,
  55,
  56,
  57,
  61,
  63,
  64,
  66,
  67,
  74,
  76,
  80,
  82,
  86,
  89,
  91,
  92,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  109,
  111,
  112,
  113,
  114,
  115,
  117,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  129,
  133,
  134,
  137,
  138,
  139,
  140,
  141,
  143,
  147,
  148,
  151,
  152,
  153,
  154,
  155,
  156,
  157,
  158,
  159,
  161,
  165,
  166,
  169,
  170,
  171,
  172,
  174,
  175,
  176,
  177,
  178,
  179,
  180,
  181,
  183,
  187,
  188,
  191,
  192,
  193,
  194,
  196,
  197,
  198,
  200,
  201,
  203,
  204,
  205,
  206,
  207,
  209,
  213,
  214,
  217,
  218,
  219,
  220,
  222,
  223,
  224,
  226,
  227,
  229,
  230,
  231,
  233,
  234,
  235,
  236,
  237,
  239,
  243,
  244,
  247,
  248,

In [25]:
testing_test(ls_list,1,2,[0,1,2],0.85,0.9,[8,11],1)

((1155, '51.9%'), (566, '35.0%', '49.0%'), 266, 43)
('price', 0.353, 0.2)
('pictures', 0.454, 0.585)
('video', 0.257, 0.45)
('zoom', 0.484, 0.57)
('size', 0.265, 0.349)
('design', 0.316, 0.581)
('battery', 0.538, 0.646)
('screen', 0.397, 0.592)
('detection', 0.082, 0.579)
('ease of use', 0.364, 0.136)
('quality', 0.214, 0.333)
('useless', 0.883, 0.548)
['price', 'pictures', 'video', 'zoom', 'size', 'design', 'battery', 'screen', 'detection', 'ease of use', 'quality', 'useless']
[[  6   2   0   4   1   2   0  11   1   0   0   3]
 [  1  83   2  16   1   7   2   2  17   1   2   8]
 [  0   5   9   0   0   0   0   1   4   0   0   1]
 [  0  12   1  45   5   0   0   0   9   0   0   7]
 [  0   7   0   1  22  14   1   2   4   1   0  11]
 [  0   0   0   0   1  18   0   1   5   1   0   5]
 [  1   2   0   1   6   0  42   2   7   2   0   2]
 [  0   0   0   1   4   6   2  29   6   0   1   0]
 [  0   2   0   2   0   0   0   0  11   1   0   3]
 [  0  24   1   3  12   1   0   1   8   8   0   1]
 [  0  

(0.519, 0.35, 0.488, 266, 43)

In [26]:
testing(test_list,1,[0,1,2],0.85,0.9,[1,1],1)

((260, '58.5%'), (131, '49.0%', '34.0%'), 21, 63)
('price', 0.286, 0.667)
('pictures', 0.415, 0.472)
('video', 0.0, 0.0)
('zoom', 0.563, 0.375)
('size', 0.6, 0.176)
('design', -1.0, 0.0)
('battery', 1.0, 0.714)
('screen', 0.778, 0.778)
('detection', -1.0, 0.0)
('ease of use', -1.0, 0.0)
('quality', 0.2, 0.25)
('useless', 0.632, 0.837)
['price', 'pictures', 'video', 'zoom', 'size', 'design', 'battery', 'screen', 'detection', 'ease of use', 'quality', 'useless']
[[  2   0   0   0   0   0   0   0   0   0   0   1]
 [  0  17   0   4   0   0   0   0   0   0   0  15]
 [  0   0   0   1   0   0   0   0   0   0   0   2]
 [  0   1   1   9   0   0   0   0   0   0   0  13]
 [  0   5   0   0   3   0   0   0   0   0   0   9]
 [  0   0   0   0   2   0   0   0   0   0   0   1]
 [  0   0   0   0   0   0   5   0   0   0   0   2]
 [  0   0   0   1   0   0   0   7   0   0   0   1]
 [  0   0   0   0   0   0   0   0   0   0   0   1]
 [  1   6   0   0   0   0   0   1   0   0   0  16]
 [  0   0   0   0   0   0

(0.585,
 0.494,
 0.336,
 21,
 63,
 [3,
  6,
  7,
  10,
  12,
  13,
  14,
  15,
  29,
  30,
  31,
  32,
  39,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  54,
  55,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  69,
  70,
  74,
  76,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  90,
  91,
  95,
  97,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  114,
  115,
  119,
  121,
  125,
  127,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  140,
  141,
  145,
  146,
  147,
  150,
  152,
  156,
  157,
  158,
  159,
  160,
  166,
  171,
  172,
  176,
  178,
  179,
  180,
  182,
  183,
  184,
  185,
  186,
  187,
  188,
  189,
  190,
  194,
  195,
  196,
  198,
  199,
  201,
  202,
  205,
  207,
  208,
  209,
  210,
  211,
  212,
  213,
  214,
  220,
  222,
  223,
  224,
  229,
  230,
  231,
  233,
  234,
  235,
  251,
  252,
  253,
  254,
  257,
  258,
  259])

In [30]:
mode=1
aspect_pattern_mode=[0,1,2]
all_list=[]
thre_useful=(0.,0.95)
thre_peak=(0.95,1.6)
M=5
N=5
thre_useful_list=np.round(thre_useful[0]+(thre_useful[1]-thre_useful[0])/M*np.arange(0,M+1),decimals=2)
thre_peak_list=np.round(thre_peak[0]+(thre_peak[1]-thre_peak[0])/N*np.arange(0,N+1),decimals=2)
matrix_correct_all=np.zeros([M+1,N+1])
matrix_useful_precision=np.zeros([M+1,N+1])
matrix_useful_recall=np.zeros([M+1,N+1])
matrix_useless_prediction=np.zeros([M+1,N+1])

for i in range(M+1):
    for j in range(N+1):
        ans=testing(ls_list,mode,aspect_pattern_mode,thre_useful_list[i],thre_peak_list[j],[0,0])
        matrix_correct_all[i,j]=ans[0]
        matrix_useful_precision[i,j]=ans[1]
        matrix_useful_recall[i,j]=ans[2]
        matrix_useless_prediction[i,j]=ans[4]        
        list_indi=((round(thre_useful_list[i],2),round(thre_peak_list[j],2)),ans)
        all_list.append(list_indi)
        print list_indi

((0.0, 0.95), (0.258, 0.234, 0.463, 589, 36))
((0.0, 1.08), (0.322, 0.248, 0.359, 589, 336))
((0.0, 1.21), (0.534, 0.415, 0.339, 589, 692))
((0.0, 1.34), (0.55, 0.467, 0.24, 589, 864))
((0.0, 1.47), (0.549, 0.548, 0.182, 589, 967))
((0.0, 1.6), (0.547, 0.612, 0.131, 589, 1034))
((0.19, 0.95), (0.266, 0.236, 0.463, 589, 45))
((0.19, 1.08), (0.329, 0.25, 0.359, 589, 344))
((0.19, 1.21), (0.534, 0.415, 0.339, 589, 692))
((0.19, 1.34), (0.55, 0.467, 0.24, 589, 864))
((0.19, 1.47), (0.549, 0.548, 0.182, 589, 967))
((0.19, 1.6), (0.547, 0.612, 0.131, 589, 1034))
((0.38, 0.95), (0.317, 0.25, 0.463, 589, 109))
((0.38, 1.08), (0.365, 0.264, 0.359, 589, 386))
((0.38, 1.21), (0.552, 0.434, 0.339, 589, 713))
((0.38, 1.34), (0.562, 0.493, 0.24, 589, 879))
((0.38, 1.47), (0.559, 0.589, 0.182, 589, 980))
((0.38, 1.6), (0.557, 0.679, 0.131, 589, 1046))
((0.57, 0.95), (0.383, 0.277, 0.459, 589, 215))
((0.57, 1.08), (0.411, 0.29, 0.355, 589, 462))
((0.57, 1.21), (0.581, 0.48, 0.336, 589, 759))
((0.57, 1

In [45]:
profile_nn_nn=(matrix_correct_all,matrix_useful_precision,matrix_useful_recall,matrix_useless_prediction)

In [39]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np 

X= thre_useful_list
Y = thre_peak_list
X, Y = np.meshgrid(X, Y)
Z = matrix_useful_recall
fig = plt.figure()
ax = fig.gca(projection='3d')
surf = ax.plot_surface(X, Y, Z, rstride=1, cstride=1, cmap='hot', linewidth=0, antialiased=False)
ax.set_zlim(0.0, 0.6)
fig.colorbar(surf, shrink=0.5, aspect=5)
plt.title('useful_recall_nn+adj')
plt.show()

KeyboardInterrupt: 